In [6]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import calendar
from modules.metrics import rmse
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Input, LSTM, ConvLSTM2D, Dense,
    Dropout, SpatialDropout2D, MaxPooling2D, BatchNormalization,
    TimeDistributed, LeakyReLU, Flatten, Average, Concatenate
)

In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
print(gpus)

1 Physical GPUs, 1 Logical GPUs
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 資料分割

In [8]:
# split data by month
def split_data_by_month(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv
    train_cases_to_exclude = {'20210116', '20210530', '20210825', '20210722', '20220904'}

    for file_name in file_list:
        datetime = file_name.split('-')[0]
        year = int(datetime[0:4])
        month = int(datetime[4:6])
        day = int(datetime[6:8])
        # 取得當月的天數
        days_in_month = calendar.monthrange(year, month)[1]
        part1_end = int(days_in_month * 0.7)
        part2_end = int(days_in_month * 0.85)

        file_path = os.path.join(data_path, file_name)

        # 個案排除，加入倒測試集
        if datetime in train_cases_to_exclude:
            test_files.append(file_path)
            continue

        if day <= part1_end:
            train_files.append(file_path)
        elif day <= part2_end:
            val_files.append(file_path)
        else:
            test_files.append(file_path)

    return (train_files, val_files, test_files)

# split data by sequence
def split_data_by_sequence(data_path):
    train_files, val_files, test_files = [], [], []
    file_list = sorted(os.listdir(data_path))  # 20210101-160505_1_160505.csv

    train_cases_to_exclude = {'20210116', '20210530', '20210825', '20210722', '20220904'}

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(data_path, file_name)
        # 加入至測試集
        if any(datetime.startswith(case) for case in train_cases_to_exclude):
            test_files.append(file_path)
            continue # 加入測試集後跳過當前迴圈，為了不將同一檔案加入訓練集或驗證集
        if datetime.startswith(('2021', '202201', '202202', '202203', '202204', '202205', '202206')):
            train_files.append(file_path)
        if datetime.startswith(('202207', '202208', '202209')):
            val_files.append(file_path)
        if datetime.startswith(('202210', '202211', '202212')):
            test_files.append(file_path)

    return (train_files, val_files, test_files)


# mont and sequence data intersect
def intersect_test_files(cells_radar_path, cells_csv_path):
    test_radar_files, test_csv_files = [], []
    file_list = sorted(os.listdir(cells_radar_path))  # 20210101-160505_1_160505.csv

    for file_name in file_list:
        datetime = file_name.split('-')[0]

        file_path = os.path.join(cells_radar_path, file_name)
        
        if not datetime.startswith(('202210', '202211', '202212')):
            continue

        datetime = file_name.split('-')[0]
        year = int(datetime[0:4])
        month = int(datetime[4:6])
        day = int(datetime[6:8])
        # 取得當月的天數
        days_in_month = calendar.monthrange(year, month)[1]
        part1_end = int(days_in_month * 0.7)
        part2_end = int(days_in_month * 0.85)
        file_path = os.path.join(cells_radar_path, file_name)

        if day > part2_end:
            test_radar_files.append(file_path)

    def make_csv_paths(radar_files):
        return [os.path.join(cells_csv_path, os.path.basename(f) + '.csv') for f in radar_files]
    test_csv_files = make_csv_paths(test_radar_files)

    return test_radar_files, test_csv_files


def get_cells_csv_files(cells_csv_path, train_radar_files, val_radar_files, test_radar_files):
    def make_csv_paths(radar_files):
        return [os.path.join(cells_csv_path, os.path.basename(f) + '.csv') for f in radar_files]

    train_cells_files = make_csv_paths(train_radar_files)
    val_cells_files = make_csv_paths(val_radar_files)
    test_cells_files = make_csv_paths(test_radar_files)

    return train_cells_files, val_cells_files, test_cells_files

# 資料生成器

In [9]:
def radar_grid_diff_processing(radar_grids):
    diff_radar_grids = []
    for i in range(len(radar_grids) - 1):
        radar_grid_diff = cv2.absdiff(radar_grids[i + 1], radar_grids[i])
        diff_radar_grids.append(radar_grid_diff)
    return np.expand_dims(np.array(diff_radar_grids), axis=-1)

def multimodal_sliding_window_generator(radar_files, csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode):
    """
    格點和CSV滑動窗口生成器，從格點資料夾中讀取格點資料，從CSV中讀取y值。
    """
    for radar_folder_path, csv_file_path in zip(radar_files, csv_files):
        # 讀取格點數據並處理差值
        radar_grids = []
        radar_grid_names = sorted(os.listdir(radar_folder_path))
        for radar_grid_name in radar_grid_names:
            radar_grid_path = os.path.join(radar_folder_path, radar_grid_name)
            radar_grid = np.load(radar_grid_path)
            radar_grid_resized = cv2.resize(radar_grid, (height, width))
            radar_grids.append(radar_grid_resized)

        diff_radar_grids = radar_grid_diff_processing(np.array(radar_grids))

        # 讀取CSV數據並計算差值
        data = pd.read_csv(csv_file_path, encoding='utf-8',
                           dtype={'fileName': str, 'day': str, 'time': str})
        data_diff = data[column_name].diff().dropna().reset_index(drop=True)

        # 提取 y 值
        lat_diff = data[column_name[0]].diff().dropna().reset_index(drop=True)
        lng_diff = data[column_name[1]].diff().dropna().reset_index(drop=True)

        # lstm 輸入
        combined_data = np.vstack([lat_diff.values, lng_diff.values]).T  # 經緯度差異合併
        
        # 使用scaler對X進行縮放
        scaled_data_diff = scaler.transform(combined_data)

        # 確保格點和CSV數據的長度相同
        if len(diff_radar_grids) != len(data_diff):
            continue

        # 生成滑動窗口
        x_lstm, x_convlstm, y_lat, y_lng = [], [], [], []
        for i in range(0, len(diff_radar_grids) - window_size, step_size):
            # 數值窗口
            x_lstm.append(scaled_data_diff[i:i + window_size])
            # 格點窗口
            x_convlstm.append(diff_radar_grids[i:i + window_size])
            # 將數值資料的y值作為標籤
            y_lat.append(lat_diff.values[i + window_size])
            y_lng.append(lng_diff.values[i + window_size])

        for x_lstm_sample, x_convlstm_sample, y_lat_sample, y_lng_sample in zip(x_lstm, x_convlstm, y_lat, y_lng):
            # 將圖像轉換為需要的數據類型和格式
            yield {
                'lstm_input': np.array(x_lstm_sample, dtype=np.float32),  # LSTM 輸入
                'convlstm_input': np.array(x_convlstm_sample, dtype=np.float32)  # ConvLSTM 輸入
            },{
                'multimodal_lat_output': np.array([y_lat_sample], dtype=np.float32),  # 緯度差標籤不縮放
                'multimodal_lng_output': np.array([y_lng_sample], dtype=np.float32)   # 經度差標籤不縮放
            }


def create_multimodal_sliding_window_dataset(radar_files, csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode):
    """
    使用滑動窗口生成多模態TensorFlow Dataset，對應LSTM和ConvLSTM輸入。
    """
    dataset = tf.data.Dataset.from_generator(
        lambda: multimodal_sliding_window_generator(
            radar_files, csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode),
        output_signature=(
            {
                'lstm_input': tf.TensorSpec(shape=(window_size, 2), dtype=tf.float32),  # LSTM 輸入
                'convlstm_input': tf.TensorSpec(shape=(window_size, height, width, channels), dtype=tf.float32)  # ConvLSTM 輸入（假設圖像是128x128，單通道）
            },
            {
                'multimodal_lat_output': tf.TensorSpec(shape=(1,), dtype=tf.float32),  # 緯度輸出
                'multimodal_lng_output': tf.TensorSpec(shape=(1,), dtype=tf.float32)   # 經度輸出
            }
        )
    )
    return dataset

In [10]:
split_data_mode = 'month'  # 'month' or 'sequence' or 'old_dataset'

cells_csv_path = r'H:\cell_data_processed\cells'
cells_radar_path = r'H:\cell_data_processed\radar\grids\global'

if split_data_mode == 'month':
    train_radar_files, val_radar_files, test_radar_files = split_data_by_month(
        cells_radar_path)
    train_csv_files, val_csv_files, test_csv_files = get_cells_csv_files(
        cells_csv_path, train_radar_files=train_radar_files, val_radar_files=val_radar_files, test_radar_files=test_radar_files)
else:
    train_radar_files, val_radar_files, test_radar_files = split_data_by_sequence(
        cells_radar_path)
    train_csv_files, val_csv_files, test_csv_files = get_cells_csv_files(
        cells_csv_path, train_radar_files=train_radar_files, val_radar_files=val_radar_files, test_radar_files=test_radar_files)

# test_radar_files, test_csv_files = intersect_test_files(cells_radar_path, cells_csv_path)

print('train_radar_files:', len(train_radar_files))
print('val_radar_files:', len(val_radar_files))
print('test_radar_files:', len(test_radar_files))
print('---' * 10)
print('train_csv_files:', len(train_csv_files))
print('val_csv_files:', len(val_csv_files))
print('test_csv_files:', len(test_csv_files))

train_radar_files: 53023
val_radar_files: 13103
test_radar_files: 14647
------------------------------
train_csv_files: 53023
val_csv_files: 13103
test_csv_files: 14647


In [11]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 2  # 窗口大小
step_size = 1  # 步長

height, width, channels = 224, 224, 1 # 格點大小為224x224, 格點

# 縮放器
scaler = joblib.load(f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 載入scaler

# 創建radar格點和CSV數據集
train_dataset = create_multimodal_sliding_window_dataset(
    train_radar_files, train_csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode='train')

val_dataset = create_multimodal_sliding_window_dataset(
    val_radar_files, val_csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode='val')

In [12]:
print(train_dataset.element_spec)
print(val_dataset.element_spec)

({'lstm_input': TensorSpec(shape=(2, 2), dtype=tf.float32, name=None), 'convlstm_input': TensorSpec(shape=(2, 224, 224, 1), dtype=tf.float32, name=None)}, {'multimodal_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'multimodal_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})
({'lstm_input': TensorSpec(shape=(2, 2), dtype=tf.float32, name=None), 'convlstm_input': TensorSpec(shape=(2, 224, 224, 1), dtype=tf.float32, name=None)}, {'multimodal_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None), 'multimodal_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})


# 建立模型

In [20]:
if split_data_mode == 'month':
    lstm_model_name = 'lstm_mt_diff2-1_e08v0.0005'
# else:  # 'sequence'
#     lstm_model_name = 'lstm_mt_diff2-1_e06v0.0011'

# lstm model
lstm_model = load_model(
    rf'weights\{split_data_mode}\lstm_multitask\{lstm_model_name}',
    custom_objects={'rmse': rmse}
)

lstm_model.summary()

Model: "lstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 2, 2)]       0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 2, 32)        4480        ['lstm_input[0][0]']             
                                                                                                  
 lstm_1-bn_1 (BatchNormalizatio  (None, 2, 32)       128         ['lstm_1[0][0]']                 
 n)                                                                                               
                                                                                                  
 lstm_1-dropout_1 (Dropout)     (None, 2, 32)        0           ['lstm_1-bn_1[0][0]

In [21]:
if split_data_mode == 'month':
    convlstm_model_name = 'convlstm_mt_diff2-1_e01v0.0014'
# else:  # 'sequence'
#     convlstm_model_name = 'convlstm_mt_diff2-1_e01v0.0018'

convlstm_model = load_model(
    rf'weights\{split_data_mode}\convlstm_multitask\{convlstm_model_name}',
    custom_objects={'rmse': rmse}
)
convlstm_model.summary()

Model: "convlstm_multi_task"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 convlstm_input (InputLayer)    [(None, 2, 224, 224  0           []                               
                                , 1)]                                                             
                                                                                                  
 convlstm_1 (ConvLSTM2D)        (None, 2, 224, 224,  38144       ['convlstm_input[0][0]']         
                                 32)                                                              
                                                                                                  
 time-distributed_1-1 (TimeDist  (None, 2, 224, 224,  128        ['convlstm_1[0][0]']             
 ributed)                        32)                                            

In [22]:
# 凍結 ConvLSTM 和 LSTM 子模型的層
for layer in convlstm_model.layers:
    layer.trainable = False

for layer in lstm_model.layers:
    layer.trainable = False

In [ ]:
# # 輸出層融合
# from tensorflow.keras.layers import Concatenate, Dense, Multiply, Subtract, Activation, Lambda, Add
# import tensorflow.keras.backend as K

# early_stopping = EarlyStopping(monitor='val_loss', patience=5,
#                                verbose=1, mode='auto', restore_best_weights=True)

# checkpoint = ModelCheckpoint(os.path.join(
#     os.getcwd(), 'weights', split_data_mode, 'multimodal_multitask_output-level_fusion', 'gate', 'multimodal_mt_diff2-1_e{epoch:02d}v{val_loss:.4f}'),
#     monitor='val_loss', save_best_only=True)

# def gated_fusion_branch(conv_output, lstm_output, name_prefix):
#     # Concatenate 輸出以利學習 gate
#     merged = Concatenate(name=f'{name_prefix}_merge')([conv_output, lstm_output])
    
#     # Gate：學習一個 0~1 的權重
#     gate = Dense(1, activation='sigmoid', name=f'{name_prefix}_gate')(merged)

#     # 1 - gate
#     one_minus_gate = Lambda(lambda x: 1.0 - x, name=f'{name_prefix}_inv_gate')(gate)

#     # gated_output = gate * conv + (1 - gate) * lstm
#     gated_conv = Multiply(name=f'{name_prefix}_conv_weighted')([gate, conv_output])
#     gated_lstm = Multiply(name=f'{name_prefix}_lstm_weighted')([one_minus_gate, lstm_output])
#     fused = Add(name=f'{name_prefix}_gated_fusion')([gated_conv, gated_lstm])

#     # 再接 Dense 輸出層
#     final_output = Dense(1, activation='linear', name=f'multimodal_{name_prefix}_output')(fused)
#     return final_output

# # 分別建立經度與緯度的門控融合
# lat_final_output = gated_fusion_branch(convlstm_model.output[0], lstm_model.output[0], 'lat')
# lng_final_output = gated_fusion_branch(convlstm_model.output[1], lstm_model.output[1], 'lng')

# # 定義門控融合模型
# gated_fusion_model = Model(
#     inputs=[convlstm_model.input, lstm_model.input],
#     outputs=[lat_final_output, lng_final_output],
#     name='multimodal_output-level_gated_fusion_model'
# )

# # 編譯模型
# gated_fusion_model.compile(optimizer=Adam(learning_rate=0.0001),
#                            loss={'multimodal_lat_output': 'mse',
#                                  'multimodal_lng_output': 'mse'},
#                            loss_weights={'multimodal_lat_output': 1.0,
#                                          'multimodal_lng_output': 1.0},
#                            metrics={'multimodal_lat_output': ['mse', rmse, 'mae'],
#                                     'multimodal_lng_output': ['mse', rmse, 'mae']})

# print(gated_fusion_model.summary())


In [24]:
# 設定訓練參數
batch_size = 4
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
train_dataset = train_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [25]:
# 開始訓練模型
history = gated_fusion_model.fit(
    train_dataset,
    validation_data=val_dataset,  # 傳入驗證集
    epochs=epochs,
    callbacks=[early_stopping, checkpoint],
    verbose=1  # 訓練過程中打印進度
)

Epoch 1/50
  46622/Unknown - 13654s 293ms/step - loss: 0.0340 - multimodal_lat_output_loss: 3.7079e-04 - multimodal_lng_output_loss: 0.0013 - multimodal_lat_output_mse: 3.7079e-04 - multimodal_lat_output_rmse: 0.0141 - multimodal_lat_output_mae: 0.0119 - multimodal_lng_output_mse: 0.0013 - multimodal_lng_output_rmse: 0.0205 - multimodal_lng_output_mae: 0.0174

INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e01v0.0016\assets


INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e01v0.0016\assets


46622/46622 [==============================] - 16956s 363ms/step - loss: 0.0340 - multimodal_lat_output_loss: 3.7079e-04 - multimodal_lng_output_loss: 0.0013 - multimodal_lat_output_mse: 3.7079e-04 - multimodal_lat_output_rmse: 0.0141 - multimodal_lat_output_mae: 0.0119 - multimodal_lng_output_mse: 0.0013 - multimodal_lng_output_rmse: 0.0205 - multimodal_lng_output_mae: 0.0174 - val_loss: 0.0016 - val_multimodal_lat_output_loss: 5.0790e-04 - val_multimodal_lng_output_loss: 0.0010 - val_multimodal_lat_output_mse: 5.0790e-04 - val_multimodal_lat_output_rmse: 0.0179 - val_multimodal_lat_output_mae: 0.0162 - val_multimodal_lng_output_mse: 0.0010 - val_multimodal_lng_output_rmse: 0.0280 - val_multimodal_lng_output_mae: 0.0263
Epoch 2/50
46622/46622 [==============================] - ETA: 0s - loss: 6.1565e-04 - multimodal_lat_output_loss: 2.2871e-04 - multimodal_lng_output_loss: 3.1705e-04 - multimodal_lat_output_mse: 2.2871e-04 - multimodal_lat_output_rmse: 0.0131 - multimodal_lat_output_m

INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e02v0.0014\assets


...

INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e04v0.0014\assets


46622/46622 [==============================] - 16458s 353ms/step - loss: 6.3283e-04 - multimodal_lat_output_loss: 2.3484e-04 - multimodal_lng_output_loss: 3.2818e-04 - multimodal_lat_output_mse: 2.3484e-04 - multimodal_lat_output_rmse: 0.0133 - multimodal_lat_output_mae: 0.0114 - multimodal_lng_output_mse: 3.2818e-04 - multimodal_lng_output_rmse: 0.0157 - multimodal_lng_output_mae: 0.0137 - val_loss: 0.0014 - val_multimodal_lat_output_loss: 4.5949e-04 - val_multimodal_lng_output_loss: 8.3746e-04 - val_multimodal_lat_output_mse: 4.5949e-04 - val_multimodal_lat_output_rmse: 0.0172 - val_multimodal_lat_output_mae: 0.0154 - val_multimodal_lng_output_mse: 8.3746e-04 - val_multimodal_lng_output_rmse: 0.0259 - val_multimodal_lng_output_mae: 0.0240
Epoch 5/50
46622/46622 [==============================] - 16378s 351ms/step - loss: 6.3533e-04 - multimodal_lat_output_loss: 2.3557e-04 - multimodal_lng_output_loss: 3.2997e-04 - multimodal_lat_output_mse: 2.3557e-04 - multimodal_lat_output_rmse: 0.

INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e06v0.0014\assets


INFO:tensorflow:Assets written to: e:\YuCheng\master_thesis\fusion\weights\month\multimodal_multitask_output-level_fusion-add_dense\gate\multimodal_mt_diff2-1_e06v0.0014\assets


46622/46622 [==============================] - 15948s 342ms/step - loss: 6.3570e-04 - multimodal_lat_output_loss: 2.3500e-04 - multimodal_lng_output_loss: 3.3092e-04 - multimodal_lat_output_mse: 2.3500e-04 - multimodal_lat_output_rmse: 0.0133 - multimodal_lat_output_mae: 0.0114 - multimodal_lng_output_mse: 3.3092e-04 - multimodal_lng_output_rmse: 0.0158 - multimodal_lng_output_mae: 0.0137 - val_loss: 0.0014 - val_multimodal_lat_output_loss: 4.6162e-04 - val_multimodal_lng_output_loss: 8.3215e-04 - val_multimodal_lat_output_mse: 4.6162e-04 - val_multimodal_lat_output_rmse: 0.0172 - val_multimodal_lat_output_mae: 0.0155 - val_multimodal_lng_output_mse: 8.3215e-04 - val_multimodal_lng_output_rmse: 0.0258 - val_multimodal_lng_output_mae: 0.0239
Epoch 7/50
46622/46622 [==============================] - 16538s 355ms/step - loss: 6.3670e-04 - multimodal_lat_output_loss: 2.3468e-04 - multimodal_lng_output_loss: 3.3222e-04 - multimodal_lat_output_mse: 2.3468e-04 - multimodal_lat_output_rmse: 0.

# 評估模型

In [13]:
column_name = ['Latitude', 'Longitude']
# 設定滑動窗口的參數
window_size = 2  # 窗口大小
step_size = 1  # 步長

height, width, channels = 224, 224, 1 # 格點大小為224x224, 格點

# 縮放器
scaler = joblib.load(f'config/{split_data_mode}/lstm_multitask_scaler.gz')  # 載入scaler

# 創建radar格點和CSV數據集
test_dataset = create_multimodal_sliding_window_dataset(
    test_radar_files, test_csv_files, column_name, window_size, step_size, height, width, channels, scaler, mode='test')

In [14]:
test_dataset.element_spec

({'lstm_input': TensorSpec(shape=(2, 2), dtype=tf.float32, name=None),
  'convlstm_input': TensorSpec(shape=(2, 224, 224, 1), dtype=tf.float32, name=None)},
 {'multimodal_lat_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
  'multimodal_lng_output': TensorSpec(shape=(1,), dtype=tf.float32, name=None)})

In [15]:
# 設定訓練參數
batch_size = 4
epochs = 50

# 使用 .batch() 和 .prefetch() 進行數據集的優化加載
test_dataset = test_dataset.batch(batch_size)\
                .prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
if split_data_mode == 'month':
    model_path = os.path.join(os.getcwd(), 'weights\month\multimodal_multitask_output-level_fusion\gate\multimodal_mt_diff2-1_e08v0.0006')
# elif split_data_mode == 'sequence':
#     model_path = os.path.join(os.getcwd(), 'weights\sequence\multimodal_multitask_output-level_fusion\multimodal_mt_diff2-1_e06v0.0012')
# else:
#     model_path = os.path.join(os.getcwd(), 'weights\old_dataset\multimodal-multitask\multimodal_mt_diff2-1_e25v0.0008')

if os.path.exists(model_path):
    output_level_fusion_model = load_model(model_path, custom_objects={'rmse': rmse})
    print('Load model successfully!')
    print(output_level_fusion_model.summary())

Load model successfully!
Model: "multimodal_output-level_gated_fusion_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 convlstm_input (InputLayer)    [(None, 2, 224, 224  0           []                               
                                , 1)]                                                             
                                                                                                  
 convlstm_1 (ConvLSTM2D)        (None, 2, 224, 224,  38144       ['convlstm_input[0][0]']         
                                 32)                                                              
                                                                                                  
 lstm_input (InputLayer)        [(None, 2, 2)]       0           []                               
                                

In [ ]:
# evaluation = output_level_fusion_model.evaluate(test_dataset)

# print("=== 經緯度差（difference）評估 ===")

# print(f'Total Loss: {evaluation[0]:.5f}')         # 總損失

# print(f'Latitude Loss: {evaluation[1]:.5f}')       # 緯度 MSE
# print(f'Longitude Loss: {evaluation[2]:.5f}')      # 經度 MSE

# print(f'Latitude MSE: {evaluation[3]:.5f}')       # 緯度 MSE
# print(f'Longitude MSE: {evaluation[6]:.5f}')      # 經度 MSE

# print(f'Latitude RMSE: {evaluation[4]:.5f}')       # 緯度 RMSE
# print(f'Longitude RMSE: {evaluation[7]:.5f}')      # 經度 RMSE

# print(f'Latitude MAE: {evaluation[5]:.5f}')       # 緯度 MAE
# print(f'Longitude MAE: {evaluation[8]:.5f}')      # 經度 MAE

# import math

# # ============= 計算誤差(公里) =============
# def calculate_mae_distance(lat_mae: float, lon_mae: float, latitude: float = 25.071182):
#     lat_km = lat_mae * 111
#     lon_km = lon_mae * 111 * math.cos(math.radians(latitude))
#     return math.sqrt(lat_km**2 + lon_km**2)

# # 範例數據：經度差 MAE = 0.02，緯度差 MAE = 0.01，五分山雷達站的緯度 = 23.5
# lat_mae = evaluation[5]
# lon_mae = evaluation[8]
# # 固定緯度
# # 五分山雷達站的經緯度
# center_lat = 25.071182
# center_lon = 121.781205

# mae_distance = calculate_mae_distance(lat_mae, lon_mae, center_lat)
# print(f"Average distance error (via MAE): {mae_distance:.2f} km")

11839/11839 [==============================] - 841s 71ms/step - loss: 5.8233e-04 - multimodal_lat_output_loss: 1.8157e-04 - multimodal_lng_output_loss: 3.3098e-04 - multimodal_lat_output_mse: 1.8157e-04 - multimodal_lat_output_rmse: 0.0116 - multimodal_lat_output_mae: 0.0097 - multimodal_lng_output_mse: 3.3098e-04 - multimodal_lng_output_rmse: 0.0167 - multimodal_lng_output_mae: 0.0148
=== 經緯度差（difference）評估 ===
Total Loss: 0.00058
Latitude Loss: 0.00018
Longitude Loss: 0.00033
Latitude MSE: 0.00018
Longitude MSE: 0.00033
Latitude RMSE: 0.01164
Longitude RMSE: 0.01675
Latitude MAE: 0.00970
Longitude MAE: 0.01476
Average distance error (via MAE): 1.83 km


In [18]:
from haversine import haversine

distances = []
lat_errors = []
lon_errors = []

for radar_folder_path, csv_file_path in zip(test_radar_files, test_csv_files):
    df = pd.read_csv(csv_file_path)
    
    # 取得經緯度真實值
    lats = df['Latitude'].values
    lons = df['Longitude'].values

    # 計算差值 (差分)
    delta_lat = np.diff(lats)
    delta_lon = np.diff(lons)
    combined_data = np.vstack([delta_lat, delta_lon]).T  # 經緯度差異合併

    # 特徵縮放
    scaled_data_diff = scaler.transform(combined_data)

    radar_grids = []
    radar_grid_names = sorted(os.listdir(radar_folder_path))
    for radar_grid_name in radar_grid_names:
        radar_grid_path = os.path.join(radar_folder_path, radar_grid_name)
        radar_grid = np.load(radar_grid_path)
        radar_grid_resized = cv2.resize(radar_grid, (height, width))
        radar_grids.append(radar_grid_resized)

    diff_radar_grids = radar_grid_diff_processing(np.array(radar_grids))

    if len(diff_radar_grids) != len(scaled_data_diff):
        with open(f'gate-error_log-{split_data_mode}.txt', 'a') as f:
            f.write(f"Length mismatch: {len(diff_radar_grids)} vs {len(scaled_data_diff)}\n")
            f.write(f"Radar folder: {radar_folder_path}\n")
            f.write(f"CSV file: {csv_file_path}\n")
            f.write(f"======================\n")
        continue

    # 準備滑動窗口資料
    lstm_inputs = []
    convlstm_inputs = []
    for i in range(len(diff_radar_grids) - window_size):
        lstm_input_sample = [scaled_data_diff[i], scaled_data_diff[i+1]]
        convlstm_input_sample = [diff_radar_grids[i], diff_radar_grids[i+1]]

        lstm_inputs.append(lstm_input_sample)
        convlstm_inputs.append(convlstm_input_sample)


    lstm_inputs = np.array(lstm_inputs)  # shape: (samples, 2, 2)
    convlstm_inputs = np.array(convlstm_inputs)  # shape: (samples, 2, 224, 224, 1)

    # 模型預測
    pred_lats_diff, pred_lons_diff = output_level_fusion_model.predict({
        'lstm_input': lstm_inputs,
        'convlstm_input': convlstm_inputs
    })

    # 預測差值還原回經緯度
    pred_lats = []
    pred_lons = []
    for i, (dlat, dlon) in enumerate(zip(pred_lats_diff, pred_lons_diff)):
        base_lat = lats[i + 2]
        base_lon = lons[i + 2]
        pred_lat = base_lat + dlat.item()
        pred_lon = base_lon + dlon.item()
        pred_lats.append(pred_lat)
        pred_lons.append(pred_lon)

    # 計算實際誤差
    for i in range(len(pred_lats)):
        real_lat = lats[i + 3]
        real_lon = lons[i + 3]
        pred_lat = pred_lats[i]
        pred_lon = pred_lons[i]

        lat_errors.append(real_lat - pred_lat)
        lon_errors.append(real_lon - pred_lon)
        distances.append(haversine((real_lat, real_lon), (pred_lat, pred_lon)))

# === 評估指標 ===
lat_errors = np.array(lat_errors)
lon_errors = np.array(lon_errors)
distances = np.array(distances)

# MAE
lat_mae = np.mean(np.abs(lat_errors))
lon_mae = np.mean(np.abs(lon_errors))

# MSE
lat_mse = np.mean(lat_errors ** 2)
lon_mse = np.mean(lon_errors ** 2)

# RMSE
lat_rmse = np.sqrt(lat_mse)
lon_rmse = np.sqrt(lon_mse)

# 結果輸出
print("=== 經緯度位置評估 ===")
print(f"Latitude MSE: {lat_mse:.6f}")
print(f"Longitude MSE: {lon_mse:.6f}")
print(f"Latitude MAE: {lat_mae:.6f}")
print(f"Longitude MAE: {lon_mae:.6f}")
print(f"Latitude RMSE: {lat_rmse:.6f}")
print(f"Longitude RMSE: {lon_rmse:.6f}")
print(f"Average Haversine distance: {np.mean(distances):.6f} km")

1/1 [==============================] - 0s 24ms/step
=== 經緯度位置評估 ===
Latitude MSE: 0.000216
Longitude MSE: 0.000339
Latitude MAE: 0.010583
Longitude MAE: 0.015003
Latitude RMSE: 0.014685
Longitude RMSE: 0.018414
Average Haversine distance: 2.112312 km


In [ ]:
# 結果輸出
print("=== 經緯度位置評估 ===")
print(f"Latitude MSE: {lat_mse:.7f}")
print(f"Longitude MSE: {lon_mse:.7f}")
print(f"Latitude MAE: {lat_mae:.7f}")
print(f"Longitude MAE: {lon_mae:.7f}")
print(f"Latitude RMSE: {lat_rmse:.7f}")
print(f"Longitude RMSE: {lon_rmse:.7f}")
print(f"Average Haversine distance: {np.mean(distances):.7f} km")